--- 
Project for the course in Microeconometrics | Summer 2021, M.Sc. Economics, Bonn University | [Aysu Avcı](https://github.com/aysuavci)

# Replication of Zimmermann (2020) <a class="tocSkip">   
---

This notebook contains my replication of the results from the following paper:

> Zimmermann, F. (2020). The Dynamics of Motivated Beliefs. American Economic Review, 110(2), 337–363.

##### Downloading and viewing this notebook:

* The best way to view this notebook is by downloading it and the repository it is located in from [GitHub](https://github.com/HumanCapitalAnalysis/template-course-project). Other viewing options like _MyBinder_ or _NBViewer_ may have issues with displaying images or coloring of certain parts (missing images can be viewed in the folder [files](https://github.com/HumanCapitalAnalysis/template-course-project/tree/master/files) on GitHub).


* The original paper, as well as the data and code provided by the authors can be accessed [here](https://www.aeaweb.org/articles?id=10.1257/aer.20180728).

##### Information about replication and individual contributions:

* For the replication, I try to remain true to the original structure of the paper so readers can easily follow along and compare. All tables and figures are named and labeled as they appear in Lindo et al. (2010).


* The tables in my replication appear transposed compared to the original tables to suit my workflow in Python.


* For transparency, all sections in the replication that constitute independent contributions by me and are not part of results presented (or include deviations from the methods used) in the paper are marked as _extensions_.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Introduction" data-toc-modified-id="1.-Introduction-1">1. Introduction</a></span></li><li><span><a href="#2.-Theoretical-Background" data-toc-modified-id="2.-Theoretical-Background-2">2. Theoretical Background</a></span></li><li><span><a href="#3.-Identification" data-toc-modified-id="3.-Identification-3">3. Identification</a></span></li><li><span><a href="#4.-Empirical-Strategy" data-toc-modified-id="4.-Empirical-Strategy-4">4. Empirical Strategy</a></span></li><li><span><a href="#5.-Replication-of-Lindo-et-al.-(2010)" data-toc-modified-id="5.-Replication-of-Lindo-et-al.-(2010)-5">5. Replication of Lindo et al. (2010)</a></span><ul class="toc-item"><li><span><a href="#5.1.-Data-&amp;-Descriptive-Statistics" data-toc-modified-id="5.1.-Data-&amp;-Descriptive-Statistics-5.1">5.1. Data &amp; Descriptive Statistics</a></span></li><li><span><a href="#5.2.-Results" data-toc-modified-id="5.2.-Results-5.2">5.2. Results</a></span><ul class="toc-item"><li><span><a href="#5.2.1.-Tests-of-the-Validity-of-the-RD-Approach" data-toc-modified-id="5.2.1.-Tests-of-the-Validity-of-the-RD-Approach-5.2.1">5.2.1. Tests of the Validity of the RD Approach</a></span></li><li><span><a href="#i.--Extension:-Visual-Validity-Check" data-toc-modified-id="i.--Extension:-Visual-Validity-Check-5.2.2">i.  Extension: Visual Validity Check</a></span></li><li><span><a href="#ii.-Advanced-Validity-Check" data-toc-modified-id="ii.-Advanced-Validity-Check-5.2.3">ii. Advanced Validity Check</a></span></li><li><span><a href="#5.2.2.-First-Year-GPAs-and-Academic-Probation" data-toc-modified-id="5.2.2.-First-Year-GPAs-and-Academic-Probation-5.2.4">5.2.2. First Year GPAs and Academic Probation</a></span></li><li><span><a href="#5.2.3.-The-Immediate-Response-to-Academic-Probation" data-toc-modified-id="5.2.3.-The-Immediate-Response-to-Academic-Probation-5.2.5">5.2.3. The Immediate Response to Academic Probation</a></span></li><li><span><a href="#5.2.4.-The-Impact-onSubsequent-Performance" data-toc-modified-id="5.2.4.-The-Impact-onSubsequent-Performance-5.2.6">5.2.4. The Impact onSubsequent Performance</a></span></li><li><span><a href="#i.-Main-Results-for-Impact-on-GPA-&amp;-Probability-of-Placing-Above-Cutoff-in-the-Next-Term" data-toc-modified-id="i.-Main-Results-for-Impact-on-GPA-&amp;-Probability-of-Placing-Above-Cutoff-in-the-Next-Term-5.2.7">i. Main Results for Impact on GPA &amp; Probability of Placing Above Cutoff in the Next Term</a></span></li><li><span><a href="#ii.-Formal-Bound-Analysis-on-Subsequent-GPA-(partial-extension)" data-toc-modified-id="ii.-Formal-Bound-Analysis-on-Subsequent-GPA-(partial-extension)-5.2.8">ii. Formal Bound Analysis on Subsequent GPA (partial extension)</a></span></li><li><span><a href="#5.2.5.-The-Impacts-on-Graduation" data-toc-modified-id="5.2.5.-The-Impacts-on-Graduation-5.2.9">5.2.5. The Impacts on Graduation</a></span></li></ul></li></ul></li><li><span><a href="#6.-Extension:-Robustness-Checks" data-toc-modified-id="6.-Extension:-Robustness-Checks-6">6. Extension: Robustness Checks</a></span><ul class="toc-item"><li><span><a href="#6.1.--A-Closer-Look-at-Students'-Subsequent-Performance." data-toc-modified-id="6.1.--A-Closer-Look-at-Students'-Subsequent-Performance.-6.1">6.1.  A Closer Look at Students' Subsequent Performance.</a></span><ul class="toc-item"><li><span><a href="#6.1.1.-Subsequent-Performance-and-Total-Credits-in-Year-2" data-toc-modified-id="6.1.1.-Subsequent-Performance-and-Total-Credits-in-Year-2-6.1.1">6.1.1. Subsequent Performance and Total Credits in Year 2</a></span></li><li><span><a href="#6.1.2.-Subsequent-Cumulative-Grade-Point-Average-(CGPA)" data-toc-modified-id="6.1.2.-Subsequent-Cumulative-Grade-Point-Average-(CGPA)-6.1.2">6.1.2. Subsequent Cumulative Grade Point Average (CGPA)</a></span></li></ul></li><li><span><a href="#6.2.-Bandwidth-Sensitivity" data-toc-modified-id="6.2.-Bandwidth-Sensitivity-6.2">6.2. Bandwidth Sensitivity</a></span></li></ul></li><li><span><a href="#7.-Conclusion" data-toc-modified-id="7.-Conclusion-7">7. Conclusion</a></span></li><li><span><a href="#8.-References" data-toc-modified-id="8.-References-8">8. References</a></span></li></ul></div>

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import pandas.io.formats.style
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm_api
import matplotlib as plt
from IPython.display import HTML

In [3]:
from auxiliary.example_project_auxiliary_predictions import *
from auxiliary.example_project_auxiliary_plots import *
from auxiliary.example_project_auxiliary_tables import *

---
# Introduction 
---

---
# 2. Theoretical Background
---


---
# 3. Identification
--- 


---
# 4. Empirical Strategy
---


---
# 5. Replication of Zimmermann (2020)
---

## 5.1. Data & Descriptive Statistics
Data Setup
* creating a variable named 'treatgroup' to group different treatments
* <span style="color:red">panel data adjustment - rank??
* generating bayesian predictions

In [4]:
df = pd.read_stata('data/data.dta')

In [5]:
#For better viewing
pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 7)
pd.set_option("display.max.rows", 30)

In [6]:
#Creating treatgroup column
tg_conditions = [
    (df['treatment'] == 'belief_announcement'),
    (df['treatment'] == 'confidence_1monthlater'),
    (df['treatment'] == 'confidence_direct_15minuteslater'),
    (df['treatment'] == 'confidence_direct_immediate'),
    (df['treatment'] == 'memory'),
    (df['treatment'] == 'memory_high'),
    (df['treatment'] == 'nofeedback'),
    (df['treatment'] == 'tournament_announcement'),
]
tg_values = [1, 2, 3, 4, 5, 6, 7, 8]
df['treatgroup'] = np.select(tg_conditions, tg_values)

In [7]:
#Generating bayesian predictions
df["post_1"] = df["prior_1"]/100

df.loc[df['pos_comparisons'] == 0, 'post_1'] = 0
df.loc[df['pos_comparisons'] == 1, 'post_1'] = 0
df.loc[df['pos_comparisons'] == 2, 'post_1'] = 0
df.loc[df['pos_comparisons'] == 3, 'post_1'] = df["prior_1"]/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_2"] = df["prior_2"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_2'] = (df["prior_2"]*(1/9)*(1/9)*(1/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_2'] = (df["prior_2"]*(24/9)*(1/9)*(1/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_2'] = (df["prior_2"]*(24/9)*(8/9)*(1/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_2'] = (df["prior_2"]*(8/9)*(8/9)*(8/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_3"] = df["prior_3"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_3'] = (df["prior_3"]*(2/9)*(2/9)*(2/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_3'] = (df["prior_3"]*(21/9)*(2/9)*(2/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_3'] = (df["prior_3"]*(21/9)*(7/9)*(2/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_3'] = (df["prior_3"]*(7/9)*(7/9)*(7/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_4"] = df["prior_4"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_4'] = (df["prior_4"]*(3/9)*(3/9)*(3/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_4'] = (df["prior_4"]*(18/9)*(3/9)*(3/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_4'] = (df["prior_4"]*(18/9)*(6/9)*(3/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_4'] = (df["prior_4"]*(6/9)*(6/9)*(6/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_5"] = df["prior_5"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_5'] = (df["prior_5"]*(4/9)*(4/9)*(4/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_5'] = (df["prior_5"]*(15/9)*(4/9)*(4/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_5'] = (df["prior_5"]*(15/9)*(5/9)*(4/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_5'] = (df["prior_5"]*(5/9)*(5/9)*(5/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_6"] = df["prior_6"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_6'] = (df["prior_6"]*(5/9)*(5/9)*(5/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_6'] = (df["prior_6"]*(12/9)*(5/9)*(5/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_6'] = (df["prior_6"]*(12/9)*(4/9)*(5/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_6'] = (df["prior_6"]*(4/9)*(4/9)*(4/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_7"] = df["prior_7"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_7'] = (df["prior_7"]*(6/9)*(6/9)*(6/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_7'] = (df["prior_7"]*(9/9)*(6/9)*(6/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_7'] = (df["prior_7"]*(9/9)*(3/9)*(6/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_7'] = (df["prior_7"]*(3/9)*(3/9)*(3/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_8"] = df["prior_8"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_8'] = (df["prior_8"]*(7/9)*(7/9)*(7/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_8'] = (df["prior_8"]*(6/9)*(7/9)*(7/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_8'] = (df["prior_8"]*(6/9)*(2/9)*(7/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_8'] = (df["prior_8"]*(2/9)*(2/9)*(2/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_9"] = df["prior_9"]/100 

df.loc[df['pos_comparisons'] == 0, 'post_9'] = (df["prior_9"]*(8/9)*(8/9)*(8/9))/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9)+ df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_9'] = (df["prior_9"]*(3/9)*(8/9)*(8/9))/(df["prior_2"]*(24/9)*(1/9)*(1/9) + df["prior_3"]*(21/9)*(2/9)*(2/9) + df["prior_4"]*(18/9)*(3/9)*(3/9) + df["prior_5"]*(15/9)*(4/9)*(4/9) + df["prior_6"]*(12/9)*(5/9)*(5/9) + df["prior_7"]*(9/9)*(6/9)*(6/9) + df["prior_8"]*(6/9)*(7/9)*(7/9) + df["prior_9"]*(3/9)*(8/9)*(8/9))
df.loc[df['pos_comparisons'] == 2, 'post_9'] = (df["prior_9"]*(3/9)*(1/9)*(8/9))/(df["prior_2"]*(24/9)*(8/9)*(1/9) + df["prior_3"]*(21/9)*(7/9)*(2/9) + df["prior_4"]*(18/9)*(6/9)*(3/9) + df["prior_5"]*(15/9)*(5/9)*(4/9) + df["prior_6"]*(12/9)*(4/9)*(5/9) + df["prior_7"]*(9/9)*(3/9)*(6/9) + df["prior_8"]*(6/9)*(2/9)*(7/9) + df["prior_9"]*(3/9)*(1/9)*(8/9))
df.loc[df['pos_comparisons'] == 3, 'post_9'] = (df["prior_9"]*(1/9)*(1/9)*(1/9))/(df["prior_1"] + df["prior_2"]*(8/9)*(8/9)*(8/9) + df["prior_3"]*(7/9)*(7/9)*(7/9) + df["prior_4"]*(6/9)*(6/9)*(6/9) + df["prior_5"]*(5/9)*(5/9)*(5/9) + df["prior_6"]*(4/9)*(4/9)*(4/9) + df["prior_7"]*(3/9)*(3/9)*(3/9) + df["prior_8"]*(2/9)*(2/9)*(2/9) + df["prior_9"]*(1/9)*(1/9)*(1/9))

df["post_10"] = df["prior_10"]/100

df.loc[df['pos_comparisons'] == 0, 'post_10'] = df["prior_10"]/(df["prior_2"]*(1/9)*(1/9)*(1/9) + df["prior_3"]*(2/9)*(2/9)*(2/9) + df["prior_4"]*(3/9)*(3/9)*(3/9) + df["prior_5"]*(4/9)*(4/9)*(4/9) + df["prior_6"]*(5/9)*(5/9)*(5/9) + df["prior_7"]*(6/9)*(6/9)*(6/9) + df["prior_8"]*(7/9)*(7/9)*(7/9) + df["prior_9"]*(8/9)*(8/9)*(8/9) + df["prior_10"])
df.loc[df['pos_comparisons'] == 1, 'post_10'] = 0
df.loc[df['pos_comparisons'] == 2, 'post_10'] = 0
df.loc[df['pos_comparisons'] == 3, 'post_10'] = 0

In [8]:
#To check the new variables
df

,treatment,rscore,performance1,performance2,performance3,performance4,performance5,performance6,performance7,performance8,performance9,wspt,donation,prior_median,prior_1,prior_2,prior_3,prior_4,prior_5,prior_6,prior_7,prior_8,prior_9,prior_10,rank,pos_comparisons,neg_comparisons,repeat_neg_comparisons,repeat_pos_comparisons,risk_investment,recall_neg_comparisons,recall_pos_comparisons,big5_1,big5_2,big5_3,big5_4,big5_5,big5_6,big5_7,big5_8,big5_9,big5_10,big5_11,big5_12,big5_13,big5_14,big5_15,gender,field_of_study,student,posterior_median,recallfeedback,qualtrics,recall_part_IQtest,recall_part_feedback,recall_parts_nonIQ,pricelist1,pricelist2,pricelist3,pricelist4,pricelist5,pricelist6,pricelist7,pricelist8,pricelist9,pricelist10,pricelist11,pricelist12,pricelist13,pricelist14,pricelist15,treatgroup,post_1,post_2,post_3,post_4,post_5,post_6,post_7,post_8,post_9,post_10
0,belief_announcement,4,1,3,4,4,5,5,5,6,8,6,1,53,2,4,9,14,24,16,13,9,7,2,7,2.0,1.0,1.0,2.0,80,1.0,2.0,1,2,4,3,2,4,3,4,2,5,3,4,3,4,4,0,Geographie,1,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0348774,0.1201635,0.2059946,0.3269755,0.1743869,0.0956403,0.0343324,0.0076294,0.0000000
1,belief_announcement,6,3,4,5,5,6,6,7,8,9,7,5,70,10,10,10,20,20,10,5,5,5,5,4,2.0,1.0,1.0,2.0,150,1.0,2.0,2,2,6,2,5,2,7,2,2,7,3,6,2,2,3,1,psychologie,1,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0910384,0.1394026,0.3072546,0.2844950,0.1137980,0.0384068,0.0199147,0.0056899,0.0000000
2,belief_announcement,8,4,6,7,7,7,8,8,9,10,5,3,85,1,4,30,30,20,9,2,2,1,1,3,3.0,0.0,0.0,2.0,80,0.0,2.0,3,5,6,3,2,3,4,4,2,2,3,2,2,3,5,1,Medienwissenschaft,1,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.032123,0.0902441,0.4534238,0.2855380,0.1101613,0.0253812,0.0023795,0.0007050,0.0000441,0.0000000
3,belief_announcement,5,4,6,7,7,7,8,8,9,10,5,0,80,3,6,6,25,40,5,5,4,3,3,9,0.0,3.0,3.0,0.0,150,3.0,0.0,1,2,7,3,2,2,6,4,2,4,1,4,1,2,3,0,Lebensmittelchemie,1,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0005947,0.0047577,0.0669045,0.2537417,0.0619487,0.1070473,0.1359897,0.1522450,0.2167707
4,belief_announcement,5,1,3,4,4,5,5,5,6,8,3,0,25,0,0,0,0,25,5,5,20,20,25,4,3.0,0.0,0.0,3.0,100,0.0,3.0,4,4,7,3,2,1,7,2,6,3,2,4,1,1,4,1,Geographie,1,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0000000,0.0000000,0.0000000,0.8311170,0.0851064,0.0359043,0.0425532,0.0053191,0.0000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,tournament_announcement,7,3,4,5,5,6,6,7,8,9,5,0,80,5,10,40,15,10,4,4,4,4,4,3,2.0,1.0,1.0,2.0,0,1.0,2.0,2,2,6,2,5,4,6,2,4,5,3,5,2,3,3,0,Wirtschaftsinformatik,1,60.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,8,0.000000,0.0814249,0.4987277,0.2061069,0.1272265,0.0407125,0.0274809,0.0142494,0.0040712,0.0000000
738,tournament_announcement,7,4,6,7,7,7,8,8,9,10,4,4,75,10,10,30,15,10,10,5,5,5,0,8,0.0,3.0,3.0,0.0,200,3.0,0.0,1,1,2,2,2,3,6,1,1,3,1,7,2,1,3,1,Lehramt Englisch/Biologie Gym/Ge,1,50.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,8,0.000000,0.0012658,0.0303797,0.0512658,0.0810127,0.1582278,0.1367089,0.2170886,0.3240506,0.0000000
739,tournament_announcement,7,4,6,7,7,7,8,8,9,10,5,0,70,2,8,20,30,10,12,8,6,3,1,6,0.0,3.0,3.0,0.0,0,3.0,0.0,3,2,5,3,4,3,5,4,3,3,3,2,3,4,4,0,Volkswirtschaftslehre,1,50.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,8,0.000000,0.0008725,0.0174501,0.0883411,0.0698004,0.1635947,0.1884611,0.2244520,0.1675210,0.0795070
740,tournament_announcement,5,4,6,7

In [9]:
#Generate posterior_median_bayes

df["posterior_median_bayes"] = (df["post_1"] + df["post_2"] + df["post_3"] + df["post_4"] + df["post_5"])*100

In [10]:
#Dummy for good/bad news (good=0, bad=1)

df["dummynews_goodbad"] = np.nan
df.loc[(df['pos_comparisons'] == 2) | (df['pos_comparisons'] == 3), 'dummynews_goodbad'] = 0
df.loc[(df['pos_comparisons'] == 0) | (df['pos_comparisons'] == 1), 'dummynews_goodbad'] = 1
df

,treatment,rscore,performance1,performance2,performance3,performance4,performance5,performance6,performance7,performance8,performance9,wspt,donation,prior_median,prior_1,prior_2,prior_3,prior_4,prior_5,prior_6,prior_7,prior_8,prior_9,prior_10,rank,pos_comparisons,neg_comparisons,repeat_neg_comparisons,repeat_pos_comparisons,risk_investment,recall_neg_comparisons,recall_pos_comparisons,big5_1,big5_2,big5_3,big5_4,big5_5,big5_6,big5_7,big5_8,big5_9,big5_10,big5_11,big5_12,big5_13,big5_14,big5_15,gender,field_of_study,student,posterior_median,recallfeedback,qualtrics,recall_part_IQtest,recall_part_feedback,recall_parts_nonIQ,pricelist1,pricelist2,pricelist3,pricelist4,pricelist5,pricelist6,pricelist7,pricelist8,pricelist9,pricelist10,pricelist11,pricelist12,pricelist13,pricelist14,pricelist15,treatgroup,post_1,post_2,post_3,post_4,post_5,post_6,post_7,post_8,post_9,post_10,posterior_median_bayes,dummynews_goodbad
0,belief_announcement,4,1,3,4,4,5,5,5,6,8,6,1,53,2,4,9,14,24,16,13,9,7,2,7,2.0,1.0,1.0,2.0,80,1.0,2.0,1,2,4,3,2,4,3,4,2,5,3,4,3,4,4,0,Geographie,1,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0348774,0.1201635,0.2059946,0.3269755,0.1743869,0.0956403,0.0343324,0.0076294,0.0000000,68.8010899,0.0
1,belief_announcement,6,3,4,5,5,6,6,7,8,9,7,5,70,10,10,10,20,20,10,5,5,5,5,4,2.0,1.0,1.0,2.0,150,1.0,2.0,2,2,6,2,5,2,7,2,2,7,3,6,2,2,3,1,psychologie,1,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0910384,0.1394026,0.3072546,0.2844950,0.1137980,0.0384068,0.0199147,0.0056899,0.0000000,82.2190612,0.0
2,belief_announcement,8,4,6,7,7,7,8,8,9,10,5,3,85,1,4,30,30,20,9,2,2,1,1,3,3.0,0.0,0.0,2.0,80,0.0,2.0,3,5,6,3,2,3,4,4,2,2,3,2,2,3,5,1,Medienwissenschaft,1,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.032123,0.0902441,0.4534238,0.2855380,0.1101613,0.0253812,0.0023795,0.0007050,0.0000441,0.0000000,97.1490262,0.0
3,belief_announcement,5,4,6,7,7,7,8,8,9,10,5,0,80,3,6,6,25,40,5,5,4,3,3,9,0.0,3.0,3.0,0.0,150,3.0,0.0,1,2,7,3,2,2,6,4,2,4,1,4,1,2,3,0,Lebensmittelchemie,1,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0005947,0.0047577,0.0669045,0.2537417,0.0619487,0.1070473,0.1359897,0.1522450,0.2167707,32.5998612,1.0
4,belief_announcement,5,1,3,4,4,5,5,5,6,8,3,0,25,0,0,0,0,25,5,5,20,20,25,4,3.0,0.0,0.0,3.0,100,0.0,3.0,4,4,7,3,2,1,7,2,6,3,2,4,1,1,4,1,Geographie,1,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0000000,0.0000000,0.0000000,0.8311170,0.0851064,0.0359043,0.0425532,0.0053191,0.0000000,83.1117021,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,tournament_announcement,7,3,4,5,5,6,6,7,8,9,5,0,80,5,10,40,15,10,4,4,4,4,4,3,2.0,1.0,1.0,2.0,0,1.0,2.0,2,2,6,2,5,4,6,2,4,5,3,5,2,3,3,0,Wirtschaftsinformatik,1,60.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,8,0.000000,0.0814249,0.4987277,0.2061069,0.1272265,0.0407125,0.0274809,0.0142494,0.0040712,0.0000000,91.3486005,0.0
738,tournament_announcement,7,4,6,7,7,7,8,8,9,10,4,4,75,10,10,30,15,10,10,5,5,5,0,8,0.0,3.0,3.0,0.0,200,3.0,0.0,1,1,2,2,2,3,6,1,1,3,1,7,2,1,3,1,Lehramt Englisch/Biologie Gym/Ge,1,50.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,8,0.000000,0.0012658,0.0303797,0.0512658,0.0810127,0.1582278,0.1367089,0.2170886,0.3240506,0.0000000,16.3924051,1.0
739,tournament_announcement,7,4,6,7,7,7,8,8,9,10,5,0,70,2,8,20,30,10,12,8,6,3,1,6,0.0,3.0,3.0,0.0,0,3.0,0.0,3,2,5,3,4,3,5,4,3,3,3,2,3,4,4,0,Volkswirtschaftslehre,1,50.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.

 <span style="color:red">CONFIDENCE COMPARISON

In [11]:
#Generate dummy for direct treatment and 1 month treatment
df["dummytreat_direct1month"] = np.nan
df.loc[(df['treatgroup'] == 3) | (df['treatgroup'] == 4), 'dummytreat_direct1month'] = 0
df.loc[df['treatgroup'] == 2, 'dummytreat_direct1month'] = 1
df

,treatment,rscore,performance1,performance2,performance3,performance4,performance5,performance6,performance7,performance8,performance9,wspt,donation,prior_median,prior_1,prior_2,prior_3,prior_4,prior_5,prior_6,prior_7,prior_8,prior_9,prior_10,rank,pos_comparisons,neg_comparisons,repeat_neg_comparisons,repeat_pos_comparisons,risk_investment,recall_neg_comparisons,recall_pos_comparisons,big5_1,big5_2,big5_3,big5_4,big5_5,big5_6,big5_7,big5_8,big5_9,big5_10,big5_11,big5_12,big5_13,big5_14,big5_15,gender,field_of_study,student,posterior_median,recallfeedback,qualtrics,recall_part_IQtest,recall_part_feedback,recall_parts_nonIQ,pricelist1,pricelist2,pricelist3,pricelist4,pricelist5,pricelist6,pricelist7,pricelist8,pricelist9,pricelist10,pricelist11,pricelist12,pricelist13,pricelist14,pricelist15,treatgroup,post_1,post_2,post_3,post_4,post_5,post_6,post_7,post_8,post_9,post_10,posterior_median_bayes,dummynews_goodbad,dummytreat_direct1month
0,belief_announcement,4,1,3,4,4,5,5,5,6,8,6,1,53,2,4,9,14,24,16,13,9,7,2,7,2.0,1.0,1.0,2.0,80,1.0,2.0,1,2,4,3,2,4,3,4,2,5,3,4,3,4,4,0,Geographie,1,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0348774,0.1201635,0.2059946,0.3269755,0.1743869,0.0956403,0.0343324,0.0076294,0.0000000,68.8010899,0.0,NaN
1,belief_announcement,6,3,4,5,5,6,6,7,8,9,7,5,70,10,10,10,20,20,10,5,5,5,5,4,2.0,1.0,1.0,2.0,150,1.0,2.0,2,2,6,2,5,2,7,2,2,7,3,6,2,2,3,1,psychologie,1,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0910384,0.1394026,0.3072546,0.2844950,0.1137980,0.0384068,0.0199147,0.0056899,0.0000000,82.2190612,0.0,NaN
2,belief_announcement,8,4,6,7,7,7,8,8,9,10,5,3,85,1,4,30,30,20,9,2,2,1,1,3,3.0,0.0,0.0,2.0,80,0.0,2.0,3,5,6,3,2,3,4,4,2,2,3,2,2,3,5,1,Medienwissenschaft,1,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.032123,0.0902441,0.4534238,0.2855380,0.1101613,0.0253812,0.0023795,0.0007050,0.0000441,0.0000000,97.1490262,0.0,NaN
3,belief_announcement,5,4,6,7,7,7,8,8,9,10,5,0,80,3,6,6,25,40,5,5,4,3,3,9,0.0,3.0,3.0,0.0,150,3.0,0.0,1,2,7,3,2,2,6,4,2,4,1,4,1,2,3,0,Lebensmittelchemie,1,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0005947,0.0047577,0.0669045,0.2537417,0.0619487,0.1070473,0.1359897,0.1522450,0.2167707,32.5998612,1.0,NaN
4,belief_announcement,5,1,3,4,4,5,5,5,6,8,3,0,25,0,0,0,0,25,5,5,20,20,25,4,3.0,0.0,0.0,3.0,100,0.0,3.0,4,4,7,3,2,1,7,2,6,3,2,4,1,1,4,1,Geographie,1,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0000000,0.0000000,0.0000000,0.8311170,0.0851064,0.0359043,0.0425532,0.0053191,0.0000000,83.1117021,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,tournament_announcement,7,3,4,5,5,6,6,7,8,9,5,0,80,5,10,40,15,10,4,4,4,4,4,3,2.0,1.0,1.0,2.0,0,1.0,2.0,2,2,6,2,5,4,6,2,4,5,3,5,2,3,3,0,Wirtschaftsinformatik,1,60.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,8,0.000000,0.0814249,0.4987277,0.2061069,0.1272265,0.0407125,0.0274809,0.0142494,0.0040712,0.0000000,91.3486005,0.0,NaN
738,tournament_announcement,7,4,6,7,7,7,8,8,9,10,4,4,75,10,10,30,15,10,10,5,5,5,0,8,0.0,3.0,3.0,0.0,200,3.0,0.0,1,1,2,2,2,3,6,1,1,3,1,7,2,1,3,1,Lehramt Englisch/Biologie Gym/Ge,1,50.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,8,0.000000,0.0012658,0.0303797,0.0512658,0.0810127,0.1582278,0.1367089,0.2170886,0.3240506,0.0000000,16.3924051,1.0,NaN
739,tournament_announcement,7,4,6,7,7,7,8,8,9,10,5,0,70,2,8,20,30,10,12,8,6,3,1,6,0.0,3.0,3.0,0.0,0,3.0,0.0,3,2,5,3,4,3,5,4,3,3,3,2,3,4,4,0,Volkswirtschaftslehre,1,50.0,NaN,Na

In [12]:
#Generate interact
df["interact_direct1month"] = df["dummytreat_direct1month"]*df["dummynews_goodbad"]

In [13]:
news_good = df[df["dummynews_goodbad"] == 0]
news_good.head()

,treatment,rscore,performance1,performance2,performance3,performance4,performance5,performance6,performance7,performance8,performance9,wspt,donation,prior_median,prior_1,prior_2,prior_3,prior_4,prior_5,prior_6,prior_7,prior_8,prior_9,prior_10,rank,pos_comparisons,neg_comparisons,repeat_neg_comparisons,repeat_pos_comparisons,risk_investment,recall_neg_comparisons,recall_pos_comparisons,big5_1,big5_2,big5_3,big5_4,big5_5,big5_6,big5_7,big5_8,big5_9,big5_10,big5_11,big5_12,big5_13,big5_14,big5_15,gender,field_of_study,student,posterior_median,recallfeedback,qualtrics,recall_part_IQtest,recall_part_feedback,recall_parts_nonIQ,pricelist1,pricelist2,pricelist3,pricelist4,pricelist5,pricelist6,pricelist7,pricelist8,pricelist9,pricelist10,pricelist11,pricelist12,pricelist13,pricelist14,pricelist15,treatgroup,post_1,post_2,post_3,post_4,post_5,post_6,post_7,post_8,post_9,post_10,posterior_median_bayes,dummynews_goodbad,dummytreat_direct1month,interact_direct1month
0,belief_announcement,4,1,3,4,4,5,5,5,6,8,6,1,53,2,4,9,14,24,16,13,9,7,2,7,2.0,1.0,1.0,2.0,80,1.0,2.0,1,2,4,3,2,4,3,4,2,5,3,4,3,4,4,0,Geographie,1,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0348774,0.1201635,0.2059946,0.3269755,0.1743869,0.0956403,0.0343324,0.0076294,0.0,68.8010899,0.0,NaN,NaN
1,belief_announcement,6,3,4,5,5,6,6,7,8,9,7,5,70,10,10,10,20,20,10,5,5,5,5,4,2.0,1.0,1.0,2.0,150,1.0,2.0,2,2,6,2,5,2,7,2,2,7,3,6,2,2,3,1,psychologie,1,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0910384,0.1394026,0.3072546,0.2844950,0.1137980,0.0384068,0.0199147,0.0056899,0.0,82.2190612,0.0,NaN,NaN
2,belief_announcement,8,4,6,7,7,7,8,8,9,10,5,3,85,1,4,30,30,20,9,2,2,1,1,3,3.0,0.0,0.0,2.0,80,0.0,2.0,3,5,6,3,2,3,4,4,2,2,3,2,2,3,5,1,Medienwissenschaft,1,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.032123,0.0902441,0.4534238,0.2855380,0.1101613,0.0253812,0.0023795,0.0007050,0.0000441,0.0,97.1490262,0.0,NaN,NaN
4,belief_announcement,5,1,3,4,4,5,5,5,6,8,3,0,25,0,0,0,0,25,5,5,20,20,25,4,3.0,0.0,0.0,3.0,100,0.0,3.0,4,4,7,3,2,1,7,2,6,3,2,4,1,1,4,1,Geographie,1,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,0.0000000,0.0000000,0.0000000,0.8311170,0.0851064,0.0359043,0.0425532,0.0053191,0.0,83.1117021,0.0,NaN,NaN
5,belief_announcement,9,4,6,7,7,7,8,8,9,10,7,10,100,20,80,0,0,0,0,0,0,0,0,3,2.0,1.0,1.0,2.0,200,1.0,2.0,2,2,4,3,2,1,3,1,2,2,2,4,1,2,3,0,bio,1,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.000000,1.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0,100.0000000,0.0,NaN,NaN


In [15]:
#Regressions for confidence (excluding the ones with rank)

import statsmodels.formula.api as sm
news_good = pd.DataFrame({"Y": news_good['posterior_median'], "D": news_good['dummytreat_direct1month']})
rslt = sm.ols(formula="Y ~ D", data=news_good).fit()
rslt.summary()


KeyError: 'posterior_median'

# 7. Conclusion
---

---
# 8. References
---